In [3]:

import sys 
sys.path.append('..')

import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import numpy as np
import seaborn as sns


In [4]:
gru_loss_name = 'gru_lead1_loss.csv'
lstm_loss_name = 'lstm_lead1_loss.csv'
ar_loss_name = 'ar_lead48_loss.csv'

In [5]:
len(np.repeat(np.arange(1,8),8))

56

forecast lead 1

In [32]:
columns_to_sum = ['test_loss', 'test_loss_true', 'test_mse', 'test_mae']
clusters = ['WEBSTER', 'HIGH', 'TED', 'HAMILTON', 'RINCONADA', 'CAMBRIDGE', 'MPL', 'BRYANT']

df_gru = pd.read_csv("../data/LossCSVs/" + gru_loss_name, index_col=0)
print(df_gru.shape)
df_lstm = pd.read_csv("../data/LossCSVs/" + lstm_loss_name, index_col=0)
print(df_lstm.shape)

df_gru['run_no'] = np.repeat(np.arange(1,8),8)
df_lstm['run_no'] = np.repeat(np.arange(1,8),8)

# We first sum each run, and then average across runs
df_gru_agg = df_gru.groupby('run_no').sum()
df_lstm_agg = df_lstm.groupby('run_no').sum()

(56, 80)
(56, 80)


,Tags,Runtime,Sweep,auto_lr_find,auto_scale_batch_size,batch_size,censor_level,censored,check_val_every_n_epoch,covariates,...,train_loss_true,train_mae,train_mse,train_rmse,trainer/global_step,val_loss,val_loss_true,val_mae,val_mse,val_rmse
run_no,,,,,,,,,,,,,,,,,,,,,
1,0.0,4351,0.0,0,0,480,16,8,8,8,...,-0.257977,6.240552,9.844394,8.596405,121200,6.668839,-0.876740,5.342267,7.608940,7.502128
2,0.0,4409,0.0,0,0,480,16,8,8,8,...,-0.262807,6.230960,9.779280,8.576250,121200,6.662349,-0.901690,5.326890,7.559557,7.505559
3,0.0,4350,0.0,0,0,480,16,8,8,8,...,-0.259166,6.248239,9.976443,8.638344,121200,6.639910,-0.823904,5.500543,8.304874,7.738717
4,0.0,4361,0.0,0,0,480,16,8,8,8,...,-0.255166,6.220175,9.825811,8.566880,121200,6.644149,-0.841963,5.480590,8.115971,7.714816
5,0.0,4457,0.0,0,0,480,16,8,8,8,...,-0.265320,6.218834,9.733888,8.559875,121200,6.661318,-0.865907,5.421925,7.942973,7.624420
6,0.0,4351,0.0,0,0,480,16,8,8,8,...,-0.278268,6.194251,9.633644,8.509918,121200,6.680645,-0.845648,5.397678,7.749438,7.582799
7,0.0,4365,0.0,0,0,480,16,8,8,8,...,-0.252555,6.229121,9.760076,8.571949,121200,6.636860,-0.880859,5.331198,7.593333,7.456200


In [42]:
print(np.round(df_gru_agg[columns_to_sum].mean(),2))
print(np.round(df_gru_agg[columns_to_sum].std(),2))

test_loss         6.56
test_loss_true   -1.17
test_mse          7.31
test_mae          4.88
dtype: float64
test_loss         0.07
test_loss_true    0.08
test_mse          0.53
test_mae          0.16
dtype: float64


In [44]:
print(np.round(df_lstm_agg[columns_to_sum].mean(),2))
print(np.round(df_lstm_agg[columns_to_sum].std(),2))

test_loss         6.84
test_loss_true   -0.71
test_mse          8.87
test_mae          5.67
dtype: float64
test_loss         0.02
test_loss_true    0.04
test_mse          0.48
test_mae          0.10
dtype: float64


forecast lead 48

In [30]:
df_ar_48 = pd.read_csv("../data/LossCSVs/" + ar_loss_name, index_col=0)
print(df_ar_48.shape)
df_ar_48 = df_ar_48[columns_to_sum]
df_ar_48['run_no'] = np.repeat(np.arange(1,8),8)
df_ar_48.head(10)

(56, 53)


,test_loss,test_loss_true,test_mse,test_mae,run_no
Name,,,,,
sunny-shape-467,1.118072,0.013835,2.201864,1.091379,1
hearty-darkness-466,1.453250,-0.078261,3.423576,1.339139,1
warm-vortex-465,1.284693,0.177268,1.560872,0.922951,1
silvery-breeze-464,0.617329,0.732744,2.409877,1.150609,1
comfy-wildflower-463,0.996154,0.939769,0.880992,0.774735,1
generous-galaxy-462,1.877385,-0.210363,9.958217,1.978139,1
lively-monkey-461,0.909120,0.835895,0.791044,0.749700,1
volcanic-serenity-460,1.483260,-0.487778,3.905221,1.380569,1
helpful-wildflower-459,1.170462,0.104843,2.998426,1.283635,2


In [33]:
df_ar_48.groupby('run_no').sum().agg(['mean', 'std'])

,test_loss,test_loss_true,test_mse,test_mae
mean,9.548014,1.679980,19.777939,8.816428
std,0.186763,0.236703,2.594258,0.382551
